In [ ]:
import geojson
import geopandas as gpd
import requests

In [ ]:
%%time
endpoint = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2015/MapServer/84/query'
s = requests.session()
s.params = {
    'where': '',
    'text': '',
    'objectIds': '',
    'time': '',
    'geometry': '[[-2336502.536,-20037700.],[1.1615796196E7,2.0656504635E7]]',
    'geometryType': 'esriGeometryEnvelope',
    'inSR': 102100, # EPSG 3857
    'spatialRel': 'esriSpatialRelIntersects',
    'relationParam': '',
    'outFields': 'STATE,COUNTY,BASENAME,NAME,STGEOMETRY',
    'returnGeometry': True,
    'returnTrueCurves': False,
    'maxAllowableOffset': '',
    'geometryPrecision': '',
    'outSR': '',
    'returnIdsOnly': False,
    'returnCountOnly': False,
    'orderByFields': '',
    'groupByFieldsForStatistics': '',
    'outStatistics': '',
    'returnZ': False,
    'returnM': False,
    'gdbVersion': '',
    'returnDistinctValues': False,
    'f': 'geojson',        
}
start = 0
done = False
features = []
crs = None
while not done:
    r = s.get(endpoint, params={
        'resultOffset': start,
        'resultRecordCount': 32,
    })
    r.raise_for_status()
    counties = geojson.loads(r.text)
    newfeats = counties.__geo_interface__['features']
    if newfeats:
        features.extend(newfeats)
        crs=counties.__geo_interface__['crs']
        start += len(newfeats)
        print("Received", len(newfeats), "entries,", start, "total")
    else:
        done = True
counties_df = gpd.GeoDataFrame.from_features(features, crs=crs)

In [ ]:
counties_df.to_file('counties.geojson', driver='GeoJSON')